# **INSTALAR E IMPORTAR BIBLIOTECAS**

In [10]:
# Instalar as bibliotecas necessárias
!pip install pandas openpyxl

# Importar as bibliotecas necessárias
import pandas as pd
from google.colab import files

# **UPLOAD ARQUIVO EXCEL**

In [11]:
# Carregar o arquivo Excel
uploaded = files.upload()

print("Arquivo carregado com sucesso!")

Saving LABAT_THE BRIDGE_CFF_BASE.xlsx to LABAT_THE BRIDGE_CFF_BASE (1).xlsx
Arquivo carregado com sucesso!


In [12]:
# Input nome planilha
nome_planilha = input("Digite o nome da planilha: ")

# Carregar o arquivo Excel
file_name = list(uploaded.keys())[0]  # Obtém o nome do arquivo carregado
df = pd.read_excel(file_name, sheet_name= nome_planilha, header=None)

# Definir a primeira linha como cabeçalho
df.columns = df.iloc[0]

# Remover a primeira linha do DataFrame
df = df.drop(df.index[0])

# Resetar o índice
df.reset_index(drop=True, inplace=True)

print("Planilha carregada com sucesso!")

Digite o nome da planilha: Relatório
Planilha carregada com sucesso!


# **TRATAMENTO DOS DADOS**

In [13]:
# Remover colunas em branco que contêm o nome 'Coluna'
cols_to_drop = [col for col in df.columns if 'Coluna' in col and df[col].isna().all()]
df = df.drop(columns=cols_to_drop)

df.head(1)

,Coluna1,Coluna2,Coluna5,Coluna6,Coluna7,Total do item,Coluna9,ago/2022,Coluna11,Coluna12,...,Coluna82,jul/2025,Coluna83,Coluna84,ago/2025,Coluna85,Coluna86,set/2025,Coluna87,Coluna88
0,1,SERVIÇOS TECNICOS,805,01/08/2022,29/08/2025,0,0,NaN,22.16,22.16,...,98.62,NaN,0.72,99.34,NaN,0.66,100,NaN,NaN,100


In [14]:
# Renomear as colunas
id_vars = ['id', 'Servico', 'duracao', 'dataInicio', 'dataFinal', 'totalCusto', '%Total']
months = df.columns[7:]

# Atribuir novas colunas
df.columns = id_vars + list(months)

In [15]:
# Identificar colunas de meses e suas categorias
value_vars = []
for i in range(7, len(df.columns) - 2, 3):
    month = df.columns[i]
    value_vars.append((month, df.columns[i], df.columns[i+1], df.columns[i+2]))

In [16]:
# Criar uma lista de dataframes para cada mês
df_list = []
for month, value, percent_month, percent_accum in value_vars:
    temp_df = df[id_vars].copy()
    temp_df['data'] = month
    temp_df['Valor'] = df[value]
    temp_df['base'] = df[percent_month]
    temp_df['% Acum'] = df[percent_accum]
    df_list.append(temp_df)

In [17]:
# Concatenar todos os dataframes na lista
final_df = pd.concat(df_list, ignore_index=True)

# Função para detectar o nível com base nos espaços em branco
def detectar_nivel(descricao):
    if pd.isna(descricao):  # Verifica se o texto é NaN (célula vazia)
        return 'Indefinido'
    espaços_em_branco = len(descricao) - len(descricao.lstrip(' '))
    if espaços_em_branco == 0:
        return 'Etapa'
    elif espaços_em_branco == 2:
        return 'Subetapa'
    elif espaços_em_branco == 4:
        return 'Tarefa'
    else:
        return 'Indefinido'

# Aplicar a função a uma nova coluna
final_df['nivel'] = final_df.iloc[:, 1].apply(detectar_nivel)  # Substitua '0' pelo índice da coluna que contém os textos

# Remover linhas onde a coluna 'nivel' é 'Indefinido'
final_df = final_df[final_df['nivel'] != 'Indefinido']

# Remover caracter "-" e espaços em branco da coluna descricao
final_df['Servico'] = final_df['Servico'].str.replace('-', '')
final_df['Servico'] = final_df['Servico'].str.strip()

# Dividir por 100 as colunas de percentual
final_df['%Total'] = final_df['%Total'].astype(float) / 100
final_df['base'] = final_df['base'].astype(float) / 100
final_df['% Acum'] = final_df['% Acum'].astype(float) / 100

final_df.head(3)

,id,Servico,duracao,dataInicio,dataFinal,totalCusto,%Total,data,Valor,base,% Acum,nivel
0,1,SERVIÇOS TECNICOS,805,01/08/2022,29/08/2025,0,0.0,ago/2022,NaN,0.2216,0.2216,Etapa
1,2,INSTALAÇÕES PROVISORIAS,805,01/08/2022,29/08/2025,0,0.0,ago/2022,NaN,0.0036,0.0036,Etapa
2,3,SERVIÇOS PRELIMINARES,128,01/02/2023,28/07/2023,0,0.0,ago/2022,NaN,NaN,NaN,Etapa


In [18]:
# Adicionar uma coluna id da obra
id_obra = input("Digite o id da obra: ")
final_df['idObra'] = id_obra

# Adicionar uma coluna id da unidade construtiva
id_und_const = input("Digite o id da unidade construtiva: ")
final_df['IDUnidConst'] = id_und_const

print('Colunas adicionadas com sucesso!')

Digite o id da obra: 166
Digite o id da unidade construtiva: 1
Colunas adicionadas com sucesso!


# **DOWNLOAD DO ARQUIVO EXCEL**

In [19]:
# Input nome do arquivo para download
nome_arquivo = input("Digite o nome do arquivo para download: ")

# Salvar a base de dados tratada em um novo arquivo Excel
output_path = f'{nome_arquivo}.xlsx'
final_df.to_excel(output_path, index=False)

# Baixar o arquivo tratado
files.download(output_path)

print("Download concluído com sucesso!")

Digite o nome do arquivo para download: LABAT_THE BRIDGE_PLAN_BASE


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download concluído com sucesso!
